In [11]:
import osmnx as ox
import networkx as nx
import numpy as np
import folium
from itertools import islice
import random
import time
import csv
import os

In [104]:
def generate_map_matrix(num_vertices, location, data_folder):
    """Generate adjacency matrix of a street network graph from OpenStreetMap data."""

    # Download the street network data from OSM
    graph = ox.graph_from_place(location, network_type='drive')

    # Ensure we have at least 100 nodes and 100 edges
    nodes = list(graph.nodes)
    edges = list(graph.edges)
    if len(nodes) < num_vertices or len(edges) < num_vertices:
        raise Exception("The graph does not have the required number of nodes or edges.")

    # Convert graph to adjacency matrix
    adj_matrix = nx.to_numpy_array(graph, nodelist=nodes)
    
    # Save the matrix to a file
    np.save(os.path.join(data_folder, 'map_matrix.npy'), adj_matrix)

    # Save node positions
    node_positions = {i: (graph.nodes[node]['y'], graph.nodes[node]['x']) for i, node in enumerate(nodes)}
    np.save(os.path.join(data_folder, 'node_positions.npy'), node_positions)

    return adj_matrix, node_positions, nodes, graph

def count_edges(adj_matrix):
    # Đếm số lượng phần tử khác không trong ma trận kề
    num_edges = np.count_nonzero(adj_matrix)
    
    # Nếu đồ thị là undirected, chia đôi số lượng cạnh
    if np.array_equal(adj_matrix, adj_matrix.T):
        num_edges //= 2

    return num_edges


def generate_interactive_map(location, data_folder):
    """Generate an interactive map of a street network graph from OpenStreetMap data to select start and end points."""

    # Download the street network data from OSM
    graph = ox.graph_from_place(location, network_type='drive')

    # Convert the graph to an undirected graph for simplicity
    graph = graph.to_undirected()

    # Create a folium map centered around District 10
    center_point = ox.geocode(location)
    folium_map = folium.Map(location=center_point, zoom_start=14)

    # Add a clickable feature to the map
    folium_map.add_child(folium.ClickForMarker())

    # Add the edges to the map
    for u, v, data in graph.edges(data=True):
        # Get the coordinates of the nodes
        u_coords = (graph.nodes[u]['y'], graph.nodes[u]['x'])
        v_coords = (graph.nodes[v]['y'], graph.nodes[v]['x'])
        
        # Draw the edge on the map
        folium.PolyLine(locations=[u_coords, v_coords], color='blue').add_to(folium_map)

    # Save the initial map
    folium_map.save(os.path.join(data_folder, 'initial_map.html'))

    print(f"Please open {data_folder}initial_map.html and click on the map to get coordinates for start and end points.")


def draw_path(node_positions, path, start_coord, end_coord, data_folder, name):
    """Draw a path on the map."""

    # Create a folium map centered around the start point
    folium_map = folium.Map(location=[start_coord[0], start_coord[1]], zoom_start=14)

    # Add start and end points to the map
    folium.CircleMarker(location=[start_coord[0], start_coord[1]], radius=5, popup='Start Point', color='black', fill=True, fill_color='white').add_to(folium_map)
    folium.Marker(location=[end_coord[0], end_coord[1]], popup='End Point', icon=folium.Icon(color='red')).add_to(folium_map)

    # Draw the path on the map
    latlons = [node_positions[node] for node in path]
    folium.PolyLine(locations=latlons, color='blue', popup=name).add_to(folium_map)

    # Save the map with the path
    folium_map.save(os.path.join(data_folder, f'{name}.html'))

    print(f"The map with {name} algorithm has been saved to {data_folder}{name}.html.")


def draw_3_shortest_path(graph, paths, start_coord, end_coord, data_folder):
    """Draw the 3 shortest paths on the map."""

    # Create a folium map centered around the start point
    folium_map = folium.Map(location=start_coord, zoom_start=14)

    # Draw the 3 shortest paths on the map
    paths = paths[::-1]
    colors = ['gray', 'lightgreen', 'blue']

    for i, path in enumerate(paths):
        nodes = list(path)
        latlons = [(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in nodes]
        folium.PolyLine(locations=latlons, color=colors[i % len(colors)], popup=f"Path {i+1}", weight=5).add_to(folium_map)

    # Add start and end points to the map
    folium.CircleMarker(location=start_coord, radius=8, popup='Start Point', color='black', fill=True, fill_color='white').add_to(folium_map)
    folium.Marker(location=end_coord, popup='End Point', icon=folium.Icon(color='red')).add_to(folium_map)

    # Save the map with paths
    folium_map.save(os.path.join(data_folder, '3_shortest_path.html'))

    print(f"The map with 3 shortest paths has been saved to {data_folder}3_shortest_path.html.")


def find_3_shortest_paths(graph, start_coord, end_coord, data_folder):
    """Find the 3 shortest paths between the start and end points."""

    # Find nearest nodes to the provided coordinates
    start_node = ox.distance.nearest_nodes(graph, start_coord[1], start_coord[0])
    end_node = ox.distance.nearest_nodes(graph, end_coord[1], end_coord[0])

    # Find the 3 shortest paths between the start and end nodes
    shortest_paths = list(islice(ox.k_shortest_paths(graph, start_node, end_node, 3, weight='length'), 3))

    # Draw the 3 shortest paths on the map
    draw_3_shortest_path(graph, shortest_paths, start_coord, end_coord, data_folder)

    return shortest_paths
    

In [87]:
location = 'District 11, Ho Chi Minh City, Vietnam'
num_vertices = 100
data_folder = './data/'
result_folder = './results/'
os.makedirs(data_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)

# Generate adjacency matrix and node positions
adj_matrix, node_positions, nodes, graph = generate_map_matrix(num_vertices, location, data_folder)

print('Number of nodes:', len(node_positions))
print('Number of edges:', count_edges(adj_matrix))

Number of nodes: 1139
Number of edges: 2764


In [71]:
# Convert adjacency matrix back to a NetworkX graph
graph = nx.from_numpy_array(adj_matrix)

# Assign original node positions to the graph
for i, node in enumerate(graph.nodes):
    graph.nodes[node]['y'] = node_positions[i][0]
    graph.nodes[node]['x'] = node_positions[i][1]

# Randomly select pairs of nodes
num_pairs = 3
node_pairs = [(random.choice(list(graph.nodes)), random.choice(list(graph.nodes))) for _ in range(num_pairs)]

# Initialize performance results
performance_results = {
    'Dijkstra': [],
    'Bellman-Ford': [],
    'Floyd-Warshall': []
}

# Measure performance for Dijkstra's algorithm
i = 0
for source, target in node_pairs:
    i += 1
    start_time = time.time()
    try:
        path = nx.dijkstra_path(graph, source, target)
        duration = time.time() - start_time
        performance_results['Dijkstra'].append(duration)

        # Get the latitude and longitude of the source and target nodes
        source_lat, source_lon = graph.nodes[source]['y'], graph.nodes[source]['x']
        target_lat, target_lon = graph.nodes[target]['y'], graph.nodes[target]['x']

        draw_path(node_positions, path, (source_lat, source_lon), (target_lat, target_lon), result_folder, f'dijkstra_{i}')
    except nx.NetworkXNoPath:
        performance_results['Dijkstra'].append(float('inf'))

# Measure performance for Bellman-Ford algorithm
i = 0
for source, target in node_pairs:
    i += 1
    start_time = time.time()
    try:
        path = nx.bellman_ford_path(graph, source, target)
        duration = time.time() - start_time
        performance_results['Bellman-Ford'].append(duration)

        # Get the latitude and longitude of the source and target nodes
        source_lat, source_lon = graph.nodes[source]['y'], graph.nodes[source]['x']
        target_lat, target_lon = graph.nodes[target]['y'], graph.nodes[target]['x']

        draw_path(node_positions, path, (source_lat, source_lon), (target_lat, target_lon), result_folder, f'bellman_ford_{i}')
    except nx.NetworkXNoPath:
        performance_results['Bellman-Ford'].append(float('inf'))

# Measure performance for Floyd-Warshall algorithm
start_time = time.time()
predecessor, distance = nx.floyd_warshall_predecessor_and_distance(graph)
floyd_warshall_duration = time.time() - start_time
i = 0
for source, target in node_pairs:
    i += 1
    if source in predecessor and target in predecessor[source]:
        performance_results['Floyd-Warshall'].append(floyd_warshall_duration)

        # Get the shortest path from source to target
        path = []
        current = target
        while current is not None:
            path.append(current)
            current = predecessor[source].get(current)
        path = path[::-1]  # Reverse path

        # Get the latitude and longitude of the source and target nodes
        source_lat, source_lon = graph.nodes[source]['y'], graph.nodes[source]['x']
        target_lat, target_lon = graph.nodes[target]['y'], graph.nodes[target]['x']

        draw_path(node_positions, path, (source_lat, source_lon), (target_lat, target_lon), result_folder, f'floyd_warshall_{i}')
    else:
        performance_results['Floyd-Warshall'].append(float('inf'))

# Print performance results
for algo, durations in performance_results.items():
    print(f"{algo} algorithm performance:")
    for i, duration in enumerate(durations):
        print(f"  Pair {i+1}: {duration:.6f} seconds")

# Save performance results to a file
file_path = os.path.join(result_folder, 'performance_results.csv')
with open(file_path, 'w', newline='') as csvfile:
    fieldnames = ['Algorithm', 'Pair', 'Duration']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for algo, durations in performance_results.items():
        for i, duration in enumerate(durations):
            writer.writerow({'Algorithm': algo, 'Pair': i+1, 'Duration': duration})

print(f"Performance results have been saved to {file_path}.")

The map with dijkstra_1 algorithm has been saved to ./resultsdijkstra_1.html.
The map with dijkstra_2 algorithm has been saved to ./resultsdijkstra_2.html.
The map with dijkstra_3 algorithm has been saved to ./resultsdijkstra_3.html.
The map with bellman_ford_1 algorithm has been saved to ./resultsbellman_ford_1.html.
The map with bellman_ford_2 algorithm has been saved to ./resultsbellman_ford_2.html.
The map with bellman_ford_3 algorithm has been saved to ./resultsbellman_ford_3.html.
The map with floyd_warshall_1 algorithm has been saved to ./resultsfloyd_warshall_1.html.
The map with floyd_warshall_2 algorithm has been saved to ./resultsfloyd_warshall_2.html.
The map with floyd_warshall_3 algorithm has been saved to ./resultsfloyd_warshall_3.html.
Dijkstra algorithm performance:
  Pair 1: 0.002537 seconds
  Pair 2: 0.004091 seconds
  Pair 3: 0.002642 seconds
Bellman-Ford algorithm performance:
  Pair 1: 0.006015 seconds
  Pair 2: 0.003760 seconds
  Pair 3: 0.004752 seconds
Floyd-Wa

In [94]:
# Generate an interactive map to select start and end points
generate_interactive_map(location, result_folder)

Please open ./results/initial_map.html and click on the map to get coordinates for start and end points.


In [105]:
start_coord = 10.7675, 106.6481
end_coord =  10.7599, 106.6484

# Find the 3 shortest paths between the start and end points
shortest_paths = find_3_shortest_paths(graph, list(start_coord), list(end_coord), result_folder)

The map with 3 shortest paths has been saved to ./results/3_shortest_path.html.
